<a href="https://colab.research.google.com/github/jatinkarthik-tripathy/RPS-Demo/blob/master/RPS_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
import os

Using TensorFlow backend.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Flatten
from tensorflow.keras.layers import Dropout, Concatenate, Bidirectional
from tensorflow.keras.models import Model
from keras import metrics


class RPS_model:
    def __init__(self, player_vocab_size, max_inp_len, max_out_len, embed_size=256):
        self.final_model = Sequential()
        self.final_model.add(Embedding(input_dim=player_vocab_size, 
                                       output_dim=embed_size,
                                       input_length=max_inp_len))
        self.final_model.add(LSTM(128, return_sequences=True))
        self.final_model.add(LSTM(128, return_sequences=True))
        self.final_model.add(LSTM(128))
        self.final_model.add(Dense(128, activation='relu'))
        self.final_model.add(Dropout(.2))
        self.final_model.add(Dense(64, activation='relu'))
        self.final_model.add(Dropout(.1))
        self.final_model.add(Dense(32, activation='relu'))
        self.final_model.add(Dense(8, activation='relu'))
        self.final_model.add(Dense(max_out_len, activation='softmax'))
        self.final_model.compile(loss="categorical_crossentropy", 
                                 optimizer="adam", 
                                 metrics=['accuracy'])
        # print(self.final_model.summary())

    def train(self, x_train, y_train, batch=100, eps=5, cbs=[]):
        self.final_model.fit(x_train, y_train,
                             batch_size=batch, epochs=eps, callbacks=cbs)

    def save_model(self, model_name):
        self.final_model.save(model_name)

    def load_model(self, model_name):
        self.final_model.load_weights(model_name)

    def test(self, p1_test, p2_test, status_test, p1_next_test, batch=100):
        score, accuracy = self.final_model.evaluate(
            [p1_test, p2_test, status_test], p1_next_test, batch_size=batch)
        print(f"Test fraction correct (NN-Score) = {score}")
        print(f"Test fraction correct (NN-Accuracy) = {accuracy}")


In [0]:
# def to_num(x):
#   if x == 'R':
#     return 1.
#   if x == 'P':
#     return 2.    
#   if x == 'S':
#     return 3.

# data_path = "/content/drive/My Drive/Models/RPS_2/train_data.txt"
# with open(data_path, 'r', encoding='utf-8') as f:
#     lines = f.read().split('\n')

# x_train = []
# y_train = []
# x_temp = []
# for line in lines[:5_000]:
#   _, p1, p2, _, p1n = line.split('\t')
#   if p1n != '_' and len(x_temp)<50:
#     x_temp.append(to_num(p1))
#     x_temp.append(to_num(p2))
#     x_train.append([x for x in x_temp])
#     y_train.append(to_num(p1n))
#   else:
#     x_temp = []

# y_train = to_categorical(y_train)

In [0]:
dataset = numpy.loadtxt("/content/drive/My Drive/Models/RPS_3/moves.csv", delimiter=",")

X = dataset[:,0:18]
Y = dataset[:,18]

Y=to_categorical(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)


In [0]:

print(X[969])
print(Y[969])

[3. 2. 1. 3. 2. 2. 3. 2. 1. 3. 2. 2. 3. 2. 1. 1. 2. 2.]
[0. 0. 0. 1.]


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle

vocab = set()
for i in [1, 2, 3]:
  vocab.add(i)
vocab = sorted(list(vocab))
vocab_size = len(vocab) + 1
max_inp_len = max([len(txt) for txt in x_train])
max_out_len = max([len(txt) for txt in y_train])
token_index = dict(
    [(char, i) for i, char in enumerate(vocab)])

print(f'Number of samples: {len(x_train)}')
print(f'Vocab size: {vocab_size}')
print(f'Max input len: {max_inp_len}')
print(f'Max output len: {max_out_len}')


print(f'len train set: {len(x_train)}')


Number of samples: 1604
Vocab size: 4
Max input len: 18
Max output len: 4
len train set: 1604


In [0]:
checkpoint_path = "/content/drive/My Drive/Ckpts/RPS3_t2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=0)

logdir = "/content/logs/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, 
                             verbose=1, mode='auto', cooldown=30, min_lr=0.0001)


old_model_name = ''
model_name = 'rps3_7.h5'
model = RPS_model(vocab_size, max_inp_len, max_out_len)
try:
  model.load_model(f"/content/drive/My Drive/Models/RPS_3/{old_model_name}")
  print(f'{old_model_name} loaded')
except:
  print('Model not trained yet')
model.train( x_train, y_train, eps=250, batch=100, 
            cbs=[tensorboard_callback, cp_callback, reduce_lr])
model.save_model(f"/content/drive/My Drive/Models/RPS_3/{model_name}")
print(f"{model_name} saved")


Model not trained yet


ValueError: ignored

In [0]:
model = tf.keras.models.load_model("/content/drive/My Drive/Models/RPS_3/rps3_5.h5")
print(model.summary())
print(f'Number of samples: {len(x_test)}')
print(f'Vocab size: {vocab_size}')
print(f'Max input len: {max_inp_len}')
print(f'Max output len: {max_out_len}')


# x_test = pad_sequences(x_test, maxlen=max_inp_len, padding='post')
# x_test, y_test = shuffle(numpy.array(x_test, dtype='float32'), 
#                            numpy.array(y_test, dtype='float32'))

results = model.evaluate(x_test, y_test)
print(f'results: {results}')